# Introduction

# Goal.
The goal of this lab is to familiarize yourself with some advanced forms of evolutionary computation (EC). In particular, you will explore the use of Evolution Strategies (ES) and Covariance Matrix Adaptation Evolution Strategies (CMA-ES). You will observe the effects of different forms of self-adaptation and how these are useful for black-box optimization.

Note once again that, unless otherwise specified, in this module's exercises we will use real-valued genotypes and that the aim of the algorithms will be to *minimize* the fitness function $f(\mathbf{x})$, i.e. lower values correspond to a better fitness!

# Exercise 1
In this exercise you will explore the use of Evolution Strategies (ES). ES are a popular class of evolutionary algorithms used for optimization problems. In particular you will explore the popular ($\mu$/$\rho$,$\lambda$)-ES where $\mu$ denotes the number of parents, $\rho\leq\mu$ the number of parents involved in the producing a single offspring (mixing number), $\lambda$ the number of offspring, and *comma* selection is employed.

To start the experiments, run the next cell. Note that, as usual, you may change the seed forthe pseudo-random number generator. 

This code will attempt to optimize the 10-dimensional __[Rosenbrock function](http://pythonhosted.org/inspyred/reference.html\#inspyred.benchmarks.Rosenbrock)__ using an Evolutionary Strategy without self-adaptation or recombination (which is very similar to the GA you used in the first module's exercises).

Try adjusting the various parameters: $\mu$, $\lambda$, and $\rho$, (by changing, respectively, `args[pop_size]`, `args[num_offspring]`, and `args[mixing_number]` in the next cell. 
- What happens if you make $\lambda$ smaller e.g. $\lambda=\mu$? You select all offsprings instead of best $\mu$ individuals
- What happens if you increase the mixing number $\rho$? Too little parents and we aren't exploiting the genetic material of the parents enough, too many parents and we lose diversity in the population
- Does this confirm or contradict the conclusions you drew in the first module?


Try out the different strategy modes (change the parameter `args[strategy_mode]`), and observe how they affect the performance of the algorithm:

- `None` means that there is no self-adaptation
- `es.GLOBAL` means each genome encodes a global step-size (mutation standard deviation)
- `es.INDIVIDUAL` means each genome encodes an independent step-size for each gene.



In [ ]:
import os
import sys

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
# mu, lambda, rho
from inspyred import benchmarks
from utils.es import GLOBAL, INDIVIDUAL
from utils.simulation import run_es_simulation

num_simulations = 30

# parameters for the ES
args = {}
args["max_generations"] = 100
args["sigma"] = 1.0  # default standard deviation
args["use_log_scale"] = True
args["num_vars"] = 10

args["pop_size"] = 20  # mu
args["num_offspring"] = 100  # lambda

# args["strategy_mode"] = None
args["strategy_mode"] = GLOBAL
# args["strategy_mode"] = INDIVIDUAL

# args["mixing_number"] = 1 #rho
args["mixing_number"] = 5
# args["mixing_number"] = 20

problem_class = benchmarks.Rosenbrock

mus = [25, 50, 100]
lambdas = [100, 200, 400]
rhos = [2, 10, 20]

for mu in mus:
    for lambda_ in lambdas:
        for rho in rhos:
            args["pop_size"] = mu
            args["num_offspring"] = lambda_
            args["mixing_number"] = rho
            args["fig_title"] = f"ES mu={mu}, lambda={lambda_}, rho={rho}"

            print(f"mu={mu}, lambda={lambda_}, rho={rho}")

            results = run_es_simulation(problem_class, num_simulations, args, False)
            print(
                f"Mean best fitness: {results.mean_best_fitness} +- {results.std_best_fitness}"
            )
            print(
                f"Mean best individual: {results.mean_best_individual} +- {results.std_best_individual}"
            )
            print("-" * 50)

In [ ]:
# strategy
from utils.simulation import plot_boxplot

num_simulations = 30

# parameters for the ES
args = {}
args["max_generations"] = 100
args["sigma"] = 1.0  # default standard deviation
args["use_log_scale"] = True
args["num_vars"] = 10

args["pop_size"] = 20  # mu
args["num_offspring"] = 100  # lambda

# args["strategy_mode"] = None
args["strategy_mode"] = GLOBAL
# args["strategy_mode"] = INDIVIDUAL

# args["mixing_number"] = 1 #rho
args["mixing_number"] = 5
# args["mixing_number"] = 20

problem_class = benchmarks.Rosenbrock

mus = [25, 50, 100]
lambdas = [100, 200, 400]
rhos = [2, 10, 20]

best_fitnesses: list[list[float]] = []
for strategy in [None, GLOBAL, INDIVIDUAL]:
    args["fig_title"] = f"ES strategy={strategy}"

    print(f"strategy={strategy}")

    results = run_es_simulation(problem_class, num_simulations, args, False)
    best_fitnesses.append(results.all_best_fitness)

    print(
        f"Mean best fitness: {results.mean_best_fitness} +- {results.std_best_fitness}"
    )
    print(
        f"Mean best individual: {results.mean_best_individual} +- {results.std_best_individual}"
    )

plot_boxplot(best_fitnesses, ["None", "Global", "Individual"], "Strategy")

# Exercise 2
In this exercise you will systematically explore some of the intuitions you gained in Exercise 1. By default this code is configured to run $10$ ES runs apiece with each different strategy mode. The best fitnesses of each run are shown in a boxplot similar to what you saw in the first module's exercises.

- How does the self-adaptation strategy influence performance on this problem?
- Does what you see here confirm what you suspected from the previous exercise?


Use the provided code as a template to systematically explore the other parameters.

**Note**: To study the effect of parameters, it's important to change only **one parameter at a time**, i.e. keep everything fixed except for the parameter you want to study, and for each parameter under study create box plots to compare different values.

**Note**: In order to make a fair comparison you must keep the number of function evaluations (i.e., $\lambda$ $\times$ `max_generations`) fixed $^{[1]}$ ).

- How do the values of $\mu$, $\rho$, and $\lambda$ influence the performance given a particular self-adaptation strategy and other parameters?
- Can you come up with any rules of thumb for choosing these parameters?


In order to see how general your results are you can explore different benchmark problems $^{[2]}$ on different numbers of variables. You can change the problem by changing the parameter `args[problem_class]` and the problem dimension (number of variables) by changing the variable `num_vars` in the next cell. Note that most benchmark problems are *scalable*, i.e. they can be defined for any number of variables.

- Can you find a choice of parameters that works properly across several problems?


---
[1]:
When you compare multiple algorithms on the same problem, the computational budget allotted to any of them should be same, in order to get a fair comparison. So, if you run experiments with different values of $\lambda$, you should adjust `max_generations` accordingly, so that the total number of evaluations is consistent across experiments. E.g.: {$\lambda=50$, `max_generations`$=100$}, {$\lambda=100$, `max_generations`$=50$}, etc. One rule of thumb sometimes used in evolutionary computation papers and black-box optimization competitions is to run each algorithm for $5000 \times n$ fitness evaluations, where $n$ is the problem dimension. Also bear in mind that some algorithms are not generational at all: in those cases, the stop criterion must be expressed in terms of absolute number of evaluations, or as a convergence condition.

[2]:
See __[link](https://pythonhosted.org/inspyred/reference.html\#single-objective-benchmarks)__ for a list of single-objective benchmark problems.

In [ ]:
# Rastrigin

num_simulations = 30

# parameters for the ES
args = {}
args["max_generations"] = 100
args["sigma"] = 1.0  # default standard deviation
args["use_log_scale"] = True
args["num_vars"] = 10

args["pop_size"] = 20  # mu
args["num_offspring"] = 100  # lambda

# args["strategy_mode"] = None
args["strategy_mode"] = GLOBAL
# args["strategy_mode"] = INDIVIDUAL

# args["mixing_number"] = 1 #rho
args["mixing_number"] = 5
# args["mixing_number"] = 20

problem_class = benchmarks.Rastrigin

mus = [25, 50, 100]
lambdas = [100, 200, 400]
rhos = [2, 10, 20]

for mu in mus:
    for lambda_ in lambdas:
        for rho in rhos:
            args["pop_size"] = mu
            args["num_offspring"] = lambda_
            args["mixing_number"] = rho
            args["fig_title"] = f"ES mu={mu}, lambda={lambda_}, rho={rho}"

            print(f"mu={mu}, lambda={lambda_}, rho={rho}")

            results = run_es_simulation(problem_class, num_simulations, args, False)
            print(
                f"Mean best fitness: {results.mean_best_fitness} +- {results.std_best_fitness}"
            )
            print(
                f"Mean best individual: {results.mean_best_individual} +- {results.std_best_individual}"
            )

In [ ]:
# Ackley

num_simulations = 30

# parameters for the ES
args = {}
args["max_generations"] = 100
args["sigma"] = 1.0  # default standard deviation
args["use_log_scale"] = True
args["num_vars"] = 10

args["pop_size"] = 20  # mu
args["num_offspring"] = 100  # lambda

# args["strategy_mode"] = None
args["strategy_mode"] = GLOBAL
# args["strategy_mode"] = INDIVIDUAL

# args["mixing_number"] = 1 #rho
args["mixing_number"] = 5
# args["mixing_number"] = 20

problem_class = benchmarks.Ackley

mus = [25, 50, 100]
lambdas = [100, 200, 400]
rhos = [2, 10, 20]

for mu in mus:
    for lambda_ in lambdas:
        for rho in rhos:
            args["pop_size"] = mu
            args["num_offspring"] = lambda_
            args["mixing_number"] = rho
            args["fig_title"] = f"ES mu={mu}, lambda={lambda_}, rho={rho}"

            print(f"mu={mu}, lambda={lambda_}, rho={rho}")

            results = run_es_simulation(problem_class, num_simulations, args, False)
            print(
                f"Mean best fitness: {results.mean_best_fitness} +- {results.std_best_fitness}"
            )
            print(
                f"Mean best individual: {results.mean_best_individual} +- {results.std_best_individual}"
            )

# Exercise 3

In this exercise you will explore the use of the Covariance Matrix Adaptation Evolution Strategy (CMA-ES). As described in the lecture CMA-ES is currently one of the most successful evolutionary optimizers. By using statistics gathered over generations it is able to adapt a covariance matrix in a completely derandomized way (in contrast to the ES we were just exploring).

While *inspyred* does not include a native implementation of CMA-ES, we have made one available to you through a similar interface as you have been using so far. An example of proper usage is provided in next cell.

Using this and the previous exercise as templates compare the performance of CMA-ES to the other Evolution Strategies you were just investigating.

- Can CMA-ES find optima to different problems with fewer function evaluations?
- How do these differences change with different pop. sizes and problem dimensions?


In [ ]:
# sphere
from utils.simulation import run_cma_simulation

num_vars = 10

# parameters for CMA-ES
args = {}
args["max_generations"] = 200
args["sigma"] = 1.0  # default standard deviation


args["pop_size"] = 20  # mu
args["num_offspring"] = 100  # lambda

# args["problem_class"] = benchmarks.Sphere
# args["problem_class"] = benchmarks.Rosenbrock
# args["problem_class"] = benchmarks.Rastrigin

generations = [50, 100, 250]
best_fitnesses: list[list[float]] = []
for generation in generations:
    args["max_generations"] = generation
    args["fig_title"] = f"CMA-ES generations={generation}"

    print(f"generations={generation}")

    results = run_cma_simulation(benchmarks.Sphere, num_simulations, args, False)
    best_fitnesses.append(results.all_best_fitness)

    print(
        f"Mean best fitness: {results.mean_best_fitness} +- {results.std_best_fitness}"
    )
    print(
        f"Mean best individual: {results.mean_best_individual} +- {results.std_best_individual}"
    )

plot_boxplot(best_fitnesses, generations, "Generations")

In [ ]:
# Rosenrock
from utils.simulation import run_cma_simulation

num_vars = 10

# parameters for CMA-ES
args = {}
args["max_generations"] = 200
args["sigma"] = 1.0  # default standard deviation


args["pop_size"] = 20  # mu
args["num_offspring"] = 100  # lambda

# args["problem_class"] = benchmarks.Sphere
# args["problem_class"] = benchmarks.Rosenbrock
# args["problem_class"] = benchmarks.Rastrigin

generations = [50, 100, 250]
best_fitnesses: list[list[float]] = []
for generation in generations:
    args["max_generations"] = generation
    args["fig_title"] = f"CMA-ES generations={generation}"

    print(f"generations={generation}")

    results = run_cma_simulation(benchmarks.Rosenbrock, num_simulations, args, False)
    best_fitnesses.append(results.all_best_fitness)

    print(
        f"Mean best fitness: {results.mean_best_fitness} +- {results.std_best_fitness}"
    )
    print(
        f"Mean best individual: {results.mean_best_individual} +- {results.std_best_individual}"
    )

plot_boxplot(best_fitnesses, generations, "Generations")

In [ ]:
# Ackely
from utils.simulation import run_cma_simulation

num_vars = 10

# parameters for CMA-ES
args = {}
args["max_generations"] = 200
args["sigma"] = 1.0  # default standard deviation


args["pop_size"] = 20  # mu
args["num_offspring"] = 100  # lambda

generations = [50, 100, 250]
best_fitnesses: list[list[float]] = []
for generation in generations:
    args["max_generations"] = generation
    args["fig_title"] = f"CMA-ES generations={generation}"

    print(f"generations={generation}")

    results = run_cma_simulation(benchmarks.Ackley, num_simulations, args, False)
    best_fitnesses.append(results.all_best_fitness)

    print(
        f"Mean best fitness: {results.mean_best_fitness} +- {results.std_best_fitness}"
    )
    print(
        f"Mean best individual: {results.mean_best_individual} +- {results.std_best_individual}"
    )

plot_boxplot(best_fitnesses, generations, "Generations")

## Instructions and questions

Concisely note down your observations from the previous exercises (follow the bullet points) and think about the following questions.

- Do the observations you made while varying μ, ρ, and λ confirm or contradict the conclusions you drew in the previous module's exercises?
- What are the advantages of self-adaptation in evolutionary computation?
- In what ways might self-adaptation be occurring in biological organisms?
- Compare the different self-adaptation strategies explored in this exercise. 
In what ways are certain strategies better than others for optimization? 
In what ways are certain strategies more biologically plausible than others?
- Describe what reasons may contribute to better performance of CMA-ES and what can be the conditions 
when CMA-ES is not better than a basic ES.